# HRRR Data Regridding and Creation

Data is downloaded from https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/hrrr_download.cgi.

In [ ]:
import requests

import xarray as xr
import netCDF4 as nc

from datetime import datetime, timedelta
import numpy as np

import cartopy
import matplotlib.pyplot as plt
import matplotlib as mpl

import xesmf as xe

In [ ]:
def radar_colormap():
    
    
    nws_reflectivity_colors = [
        "#ccffff", # -30
        "#cc99cc", # -25
        "#996699", # -20
        "#663366", # -15
        "#cccc99", # -10
        "#999966", # -5
        "#646464", # 0
        "#04e9e7", # 5
        "#019ff4", # 10
        "#0300f4", # 15
        "#02fd02", # 20
        "#01c501", # 25
        "#008e00", # 30
        "#fdf802", # 35
        "#e5bc00", # 40
        "#fd9500", # 45
        "#fd0000", # 50
        "#d40000", # 55
        "#bc0000", # 60
        "#f800fd", # 65
        "#9854c6", # 70
        "#fdfdfd" # 75
        ]

    return mpl.colors.ListedColormap(nws_reflectivity_colors)

new_cmap = radar_colormap()

In [ ]:
#Create a datetime object to grab 
datetime_object = datetime(2022, 11, 4, 21)
forecast_hour = 5
runtime = datetime_object - timedelta(hours=forecast_hour)

In [ ]:
#open both HRRR datasets to regrid
ds = xr.open_dataset(f'/home/scratch/jcorner1/UtahHRRR/hrrr_{datetime_object.strftime("%Y%m%d")}_t{runtime.strftime("%H")}f{str(forecast_hour).zfill(2)}.grib2', engine = 'cfgrib', filter_by_keys={'typeOfLevel': 'atmosphere'})
ds_hrrr = xr.open_dataset('hrrr_esrl_2019052800f027.grib2', filter_by_keys={'typeOfLevel': 'atmosphere'})

## Regrid the Data

In [ ]:
#Create the regridder.
regridder = xe.Regridder(ds, ds_hrrr, "bilinear")
regridder

In [ ]:
#regrid the data
ds_out = regridder(ds)
ds_out

## Create a New Dataset for the Regridded data

In [ ]:
#subset different parts of the new file
lon = ds_out.longitude.values
lat = ds_out.latitude.values
time = ds_out.time.values
refc = ds_out.refc.values

In [ ]:
#Create varibales/coordinates of the new file 
new_ds = xr.Dataset(

        data_vars=dict(
        refc=(["x", "y"], refc))
        
        ,coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
        run_time=time,
        valid_time=datetime_object.strftime("%Y-%m-%dT%H:00"),
    ))

In [ ]:
#Save all the data off to the new file.
new_ds.to_netcdf(f'hrrr_{datetime_object.strftime("%Y%m%d")}_t{runtime.strftime("%H")}f{str(forecast_hour).zfill(2)}.nc4')
#new_ds.to_netcdf(f'/home/scratch/jcorner1/UtahHRRR/hrrr_{datetime_object.strftime("%Y%m%d")}_t{runtime.strftime("%H")}f{str(forecast_hour).zfill(2)}.nc4')

In [ ]:
#open the new dataset
dataset = xr.open_dataset(f'hrrr_{datetime_object.strftime("%Y%m%d")}_t{runtime.strftime("%H")}f{str(forecast_hour).zfill(2)}.nc4')
dataset

In [ ]:
#plot the new data
fig = plt.figure()
fig.set_size_inches(16,12)
ax1 = fig.add_subplot(111, projection = cartopy.crs.LambertConformal(central_latitude = 40., central_longitude = -105.))


ax1.add_feature(cartopy.feature.STATES.with_scale('50m'), facecolor='none', zorder = 9)
ax1.add_feature(cartopy.feature.OCEAN)
ax1.add_feature(cartopy.feature.COASTLINE)
ax1.add_feature(cartopy.feature.BORDERS)

ax1.set_extent([-122,-77,20,50])
norm = mpl.colors.Normalize(vmin=-30, vmax=80)
ax1.pcolormesh(dataset.lon.values, dataset.lat.values, dataset.refc.values, transform=cartopy.crs.PlateCarree(), shading='auto', norm = norm, cmap= new_cmap)